In [4]:
# ===============================
#   PART 0 — Install libraries
# ===============================
!pip install numpy pandas scikit-learn tensorflow matplotlib seaborn

# ===============================
#   PART 1 — Load NSL-KDD
# ===============================
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
# URLs of NSL-KDD files
train_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
test_url  = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

cols = [
    'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
    'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
    'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login',
    'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate',
    'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count',
    'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
    'dst_host_srv_rerror_rate','label', 'difficulty_score' # Added 'difficulty_score' to account for all 43 columns
]

train = pd.read_csv(train_url, names=cols)
test  = pd.read_csv(test_url, names=cols)

# Drop the difficulty_score column as it's not a feature for the model
train = train.drop(columns=['difficulty_score'])
test = test.drop(columns=['difficulty_score'])

# Convert labels to binary
train['label'] = train['label'].apply(lambda x: 0 if x == 'normal' else 1)
test['label']  = test['label'].apply(lambda x: 0 if x == 'normal' else 1)

# One-hot encode categorical
cat = ['protocol_type','service','flag']
train = pd.get_dummies(train, columns=cat)
test = pd.get_dummies(test, columns=cat)

# Align columns so train and test have the same set of features
train_encoded, test_encoded = train.align(test, join='left', axis=1, fill_value=0)

# Now both DataFrames have identical columns
print("Train shape:", train_encoded.shape)
print("Test shape:", test_encoded.shape)

# test all numeric or not
#print(train_encoded.select_dtypes(include=['object']).head())
#print(test_encoded.select_dtypes(include=['object']).head())

# divide data to xtrain and ytrain
X_train = train_encoded.drop('label', axis=1).astype(float)
y_train = train_encoded['label']

X_test = test_encoded.drop('label', axis=1).astype(float)
y_test = test_encoded['label']

# Now both DataFrames have identical columns
print("Train shape:", train_encoded.shape)
print("Test shape:", test_encoded.shape)

# ===============================
#   PART 2 — Build IDS model (MLP)
# ===============================

# def build_ids(input_dim):
#     model = models.Sequential([
#         layers.Dense(64, activation='relu', input_shape=(input_dim,)),
#         layers.Dense(32, activation='relu'),
#         layers.Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# ids = build_ids(X_train.shape[1])
# ids.fit(X_train, y_train, epochs=5, batch_size=256, validation_split=0.1)
# loss, accuracy = ids.evaluate(X_test, y_test)
# print(f"Test accuracy: {accuracy}")


# ----------------------------
# 1. Reshape input for LSTM
# ----------------------------
# LSTM expects: (batch, timesteps, features)
# We treat each sample as 1 timestep with all features
X_train_lstm = np.expand_dims(X_train, axis=1)   # shape becomes (N, 1, features)
X_test_lstm  = np.expand_dims(X_test, axis=1)

# ----------------------------
# 2. Build LSTM-based IDS
# ----------------------------
def build_ids_lstm(input_dim):
    model = models.Sequential([
        layers.LSTM(64, input_shape=(1, input_dim), return_sequences=False),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

ids = build_ids_lstm(X_train.shape[1])

# ----------------------------
# 3. Train the model
# ----------------------------
ids.fit(X_train_lstm, y_train,
        epochs=5,
        batch_size=256,
        validation_split=0.1)

# ----------------------------
# 4. Evaluate
# ----------------------------
loss, accuracy = ids.evaluate(X_test_lstm, y_test)
print(f"Test accuracy: {accuracy}")



# ===============================
#   PART 3 — Generate Adversarial Examples (FGSM) using feature space
# ===============================

# FGSM attack
# def fgsm_attack(model, x, y, eps=0.1):
#     x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
#     y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

#     # Reshape y_tensor to match the output shape of the model (batch_size, 1)
#     y_tensor = tf.expand_dims(y_tensor, axis=-1)

#     with tf.GradientTape() as tape:
#         tape.watch(x_tensor)
#         pred = model(x_tensor)
#         loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

#     grad = tape.gradient(loss, x_tensor)
#     ae = x_tensor + eps * tf.sign(grad)
#     return np.clip(ae.numpy(), 0, 1)

# # Generate 5k adversarial samples
# X_adv = fgsm_attack(ids, X_train[:5000], y_train[:5000])
# y_adv = y_train[:5000].copy()
# #show which features affected by FGSM
# diff = X_adv - X_train[:5000]
# changed_features = (diff != 0).sum(axis=0)
# print(changed_features)

# ===============================
#   PART 3 — Generate Adversarial Examples (FGSM) using problem space
# ===============================
# ========== 1) Features allowed to be modified ==========
# modifiable_features = [
#     'duration','src_bytes','srv_count',
#     'count','dst_host_count','dst_host_srv_count'
# ]

# p = 0.75 # = 5% max change per feature (problem space)

# # copy of data before encoding
# raw_test = pd.read_csv(test_url, names=cols)
# raw_test = raw_test.drop(columns=['difficulty_score'])
# raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)

# # ========== 2) FGSM on encoded space but apply to raw features ==========
# def problem_space_fgsm(model, X_raw, X_encoded, y, eps=0.1):

#     # convert int tensor
#     X_raw = X_raw.copy()
#     X = tf.convert_to_tensor(X_encoded, dtype=tf.float32)
#     y = tf.convert_to_tensor(y, dtype=tf.float32)
#     y = tf.expand_dims(y, axis=-1)

#     with tf.GradientTape() as tape:
#         tape.watch(X)
#         pred = model(X)
#         loss = tf.keras.losses.binary_crossentropy(y, pred)

#     grad = tape.gradient(loss, X)

#     # identify gradient of features
#     grad_sign = np.sign(grad)

#     # ========== 3) modify only modifiable features  ==========
#     for f in modifiable_features:
#         g = grad_sign[:, X_test.columns.get_loc(f)]

#         # أقصى تغير مسموح به في problem-space p%
#         delta = p * X_raw[f].abs()

#         # القيمة الجديدة
#         X_raw[f] = X_raw[f] + g * delta

#         # ضمان عدم السالب
#         X_raw[f] = np.clip(X_raw[f], 0, None)

#     # ========== 4) apply One-hot ==========
#     adv = pd.get_dummies(X_raw, columns=['protocol_type','service','flag'])

#     # align مع train
#     adv_encoded = adv.reindex(columns=test_encoded.columns, fill_value=0)

#     # فصل X و y
#     X_adv = adv_encoded.drop('label', axis=1).astype(float)
#     y_adv = adv_encoded['label']

#     return X_adv.values, y_adv.values


# # # Generate 5k adversarial samples
# X_raw_subset = raw_test.iloc[:5000].reset_index(drop=True)
# X_encoded_subset = X_test.iloc[:5000].values
# y_subset = y_test.iloc[:5000].values

# # Generate adversarial samples for ALL test data
# # X_raw_subset = raw_test.reset_index(drop=True)
# # X_encoded_subset = X_test.values
# # y_subset = y_test.values

# X_adv, y_adv = problem_space_fgsm(ids, X_raw_subset, X_encoded_subset, y_subset, eps=0.1)

# print("Generated adversarial examples (problem space):", X_adv.shape)

# #show which features affected by FGSM
# diff = X_adv - X_test[:5000]
# changed_features = (diff != 0).sum(axis=0)
# print(changed_features)

# ===============================
#   PART 3 —  an other way to Generate Adversarial Examples (FGSM) using problem space
# ===============================
# ===============================
# FGSM EXACTLY LIKE MANDA (2024)
# ===============================



# -------- 1) Problem-space numeric features only --------
numeric_features = [
    'duration','src_bytes','count','srv_count','dst_host_count','dst_host_srv_count'
]

# -------- 2) All categorical that must NOT change --------
categorical_features = ['protocol_type','service','flag']
# copy of data before encoding
raw_test = pd.read_csv(test_url, names=cols)
raw_test = raw_test.drop(columns=['difficulty_score'])
raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)
X_raw_41 = raw_test.drop('label', axis=1)
# X_encoded_121 = pd.get_dummies(X_raw_41, columns=categorical_features)
# X_train_encoded_cols = X_encoded_121.columns

def manda_fgsm(model, X_raw_unencoded, X_encoded_aligned, y_labels, eps=0.1, model_expected_cols=None,p=0.75):

    # ---------- Step 1: FGSM in feature-space ----------
    X = tf.convert_to_tensor(X_encoded_aligned, dtype=tf.float32)
    X = tf.expand_dims(X, axis=1) # Add the timesteps dimension for LSTM
    y_tensor = tf.convert_to_tensor(y_labels, dtype=tf.float32)
    y_tensor = tf.expand_dims(y_tensor, axis=-1)

    with tf.GradientTape() as tape:
        tape.watch(X)
        pred = model(X)
        loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

    grad = tape.gradient(loss, X).numpy()
    grad_sign = np.sign(grad)

    # Ensure model_expected_cols is provided
    if model_expected_cols is None:
        raise ValueError("model_expected_cols must be provided for problem-space FGSM")

    # Nullify perturbations on categorical / non-diff features
    # Iterate through categorical features and find their one-hot encoded columns in model_expected_cols
    for cat in categorical_features:
        # Find indices of one-hot encoded columns corresponding to the categorical feature
        col_indices = [i for i, c in enumerate(model_expected_cols) if c.startswith(cat+'_')]
        if col_indices: # Only modify if such columns exist
            grad_sign[:, 0, col_indices] = 0 # Adjust index for 3D grad_sign

    # ---------- Step 2: Map back to problem-space ----------
    X_raw_adv = X_raw_unencoded.copy()

    for f in numeric_features:
        # Check if the numeric feature exists in the model's expected columns
        if f in model_expected_cols:
            # Apply perturbation to the raw feature based on the gradient of its encoded counterpart
            # The gradient sign for 'f' is at col_index_in_encoded_aligned in grad_sign
            idx = model_expected_cols.get_loc(f)
            g = grad_sign[:, 0, idx]               # FGSM direction (+1 / -1) - Adjusted for 3D grad_sign

            delta = p * X_raw_adv[f].abs()     # allowed change = p%

            # apply modification
            X_raw_adv[f] = X_raw_adv[f] +  (eps * g * delta)
            # X_raw_adv[f] = X_raw_adv[f] + eps * grad_sign[:, idx] * np.abs(X_raw_unencoded[f])
            X_raw_adv[f] = np.clip(X_raw_adv[f], 0, None) # Ensure non-negative

    # Re-encode categorical features from the modified raw data
    adv = pd.get_dummies(X_raw_adv, columns=categorical_features)

    # Align with model's expected columns (X_train.columns)
    X_adv_encoded = adv.reindex(columns=model_expected_cols, fill_value=0)

    # Separate X and y (y labels remain unchanged, as it's an adversarial attack on X)
    X_final = X_adv_encoded.astype(float).values # Removed .drop('label', axis=1)
    y_final = y_labels # Labels remain the same

    return X_final, y_final


# # # Generate 5k adversarial samples
X_raw_subset = X_raw_41.iloc[:200].reset_index(drop=True)
X_encoded_subset = X_test.iloc[:200].values # Use the correctly aligned X_test data
y_subset = y_test.iloc[:200].values

X_adv, y_adv = manda_fgsm(ids, X_raw_subset, X_encoded_subset, y_subset, eps=0.1, model_expected_cols=X_train.columns,p=0.075) # Pass X_train.columns for alignment

print("Generated adversarial examples (problem space):", X_adv.shape)

#show which features affected by FGSM
diff = pd.DataFrame(X_adv, columns=X_train.columns) - X_test.iloc[:200] # Convert X_adv to DataFrame with correct columns for comparison
changed_features = (diff != 0).sum(axis=0)
print(changed_features)

# ===============================
#   PART 4 — Compute MANIFOLD SCORE
# ===============================

# PCA projection distance as manifold score
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

pca = PCA(n_components=10)
pca.fit(X_train)

def manifold_score(x):
    proj = pca.inverse_transform(pca.transform(x))
    return np.mean((x - proj)**2, axis=1)

manifold_clean = manifold_score(X_train[:200])
manifold_adv   = manifold_score(X_adv)


# ===============================
#   PART 5 — Compute DB SCORE (Deep Boundary)
# ===============================

def db_score(model, x):
    # Reshape input for the LSTM model
    x_reshaped = np.expand_dims(x, axis=1)
    with tf.GradientTape() as tape:
        x_t = tf.convert_to_tensor(x_reshaped, dtype=tf.float32)
        tape.watch(x_t)
        pred = model(x_t)
    grad = tape.gradient(pred, x_t).numpy()
    # The gradient will also be 3D (batch, 1, features), so squeeze the middle dimension
    return np.mean(np.abs(grad[:, 0, :]), axis=1)

db_clean = db_score(ids, X_train[:200])
db_adv   = db_score(ids, X_adv)


# ===============================
#   PART 6 — Build MANDA dataset
# ===============================

S1 = np.concatenate([manifold_clean, manifold_adv])
S2 = np.concatenate([db_clean, db_adv])
Y  = np.concatenate([np.zeros_like(manifold_clean), np.ones_like(manifold_adv)])

df_manda = pd.DataFrame({'manifold': S1, 'db': S2, 'label': Y})
df_manda.head()
#print(df_manda)

# ===============================
#   PART 7 — Train MANDA (Logistic Regression)
# ===============================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

clf = LogisticRegression()
clf.fit(df_manda[['manifold', 'db']], df_manda['label'])

pred = clf.predict(df_manda[['manifold', 'db']])
print("MANDA accuracy:", accuracy_score(df_manda['label'], pred))
print("AUC:", roc_auc_score(df_manda['label'], pred))

# ===============================
#   PART 8 — Fixing FPR at 5% or 15%
# ===============================

from sklearn.metrics import roc_curve

scores = clf.predict_proba(df_manda[['manifold', 'db']])[:,1]
fpr, tpr, th = roc_curve(df_manda['label'], scores)

def get_threshold(target_fpr):
    idx = np.argmin(np.abs(fpr - target_fpr))
    return th[idx]

thr_5  = get_threshold(0.05)
thr_15 = get_threshold(0.15)

print("Threshold at 5% FPR:", thr_5)
print("Threshold at 15% FPR:", thr_15)


Train shape: (125973, 123)
Test shape: (22544, 123)
Train shape: (125973, 123)
Test shape: (22544, 123)
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


443/443 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9292 - loss: 0.2274 - val_accuracy: 0.9679 - val_loss: 0.0680
Epoch 2/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9763 - loss: 0.0604 - val_accuracy: 0.9849 - val_loss: 0.0522
Epoch 3/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9854 - loss: 0.0434 - val_accuracy: 0.9709 - val_loss: 0.0622
Epoch 4/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9799 - loss: 0.0514 - val_accuracy: 0.9884 - val_loss: 0.0394
Epoch 5/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9875 - loss: 0.0396 - val_accuracy: 0.9807 - val_loss: 0.0490
705/705 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7823 - loss: 1.1526
Test accuracy: 0.7781671285629272
Generated adversarial examples (problem space): (200, 122)
duration           23
src_bytes         106
dst_bytes           0
land                0
wrong_fragment      0
                 ... 
flag_S1             0
flag_S2             0
flag_S3             0
flag_SF  

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
